In [8]:
import numpy as np

In [7]:
L = 5
delta_x = L/10

In [23]:
dim_A = (L-2)**2
A = np.zeros((dim_A, dim_A))
for i in range(dim_A):
    A[i,i] = -4
    if i+1<dim_A:
        A[i+1, i] = 1
        A[i, i+1] = 1
    if L-2+i < dim_A:
        A[L-2+i, i] = 1
        A[i, L-2+i] = 1
for m in range(1,L-2):
        A[m*(L-2), m*(L-2)-1] = 0
        A[m*(L-2)-1, m*(L-2)] = 0
print(A)

[[-4.  1.  0.  1.  0.  0.  0.  0.  0.]
 [ 1. -4.  1.  0.  1.  0.  0.  0.  0.]
 [ 0.  1. -4.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0. -4.  1.  0.  1.  0.  0.]
 [ 0.  1.  0.  1. -4.  1.  0.  1.  0.]
 [ 0.  0.  1.  0.  1. -4.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.  0. -4.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.  1. -4.  1.]
 [ 0.  0.  0.  0.  0.  1.  0.  1. -4.]]
